In [24]:
!pip install huggingface_hub

In [1]:
from tqdm import tqdm

import random

import gensim
from gensim.models import Word2Vec

import re

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from transformers import pipeline

import numpy as np
import pandas as pd

import huggingface_hub

Откроем получившийся файлик и прочитаем из него информацию

In [3]:
with open('texts_tolstoy.txt') as file:
    texts_tolstoy = file.read()

Разделим файлик по текстам при помощи переноса на новую строку

In [4]:
texts = texts_tolstoy.split('\n')

Подготовим тексты для модели:

каждый текст разобьем на слова (то есть каждый текст -- список слов)

In [5]:
texts_w2v = []
for text in texts:
    text = text.split()
    texts_w2v.append(text)

Обучаем w2v модельку на наших данных!

Я честно хотела взять все тексты (их на сайте около 200), но моделька постоянно вылетала от такого объема... 

*(это можно считать перспективой на будущее -- обучить что-то побольше!)*

In [6]:
%time model = gensim.models.Word2Vec(texts_w2v)

CPU times: total: 4.36 s
Wall time: 5.33 s


Просто посмотреть на словарик и его длину

In [7]:
vocab = [w for w in model.wv.key_to_index]
len(vocab)

52273

Делим тексты, которые скачивали, на предложения (по знакам препинания . ! и ?)

In [8]:
sentences_all = re.split(r'(?<=\w[.!?]) ', texts_tolstoy)

Случайные 5000 предложений

In [9]:
sentences = random.choices(sentences_all, k=5000)

Определяем функцию для "генерации" предложений по предложению Толстого.

Выбираем рандомные индексы слов в предложении, которые будем заменять

Если рандом решил, что мы меняем слово, находим самое близкое к заданному. А еще пробуем поменять его форму на ту же, что была у исходного слова. Если не получается, так уж и быть, оставляем. Выводим итоговое предложение

In [10]:
def generate_text(sentence):
    ready_sentence = []
    # случайные индексы 0 и 1 для всех слов в предложении
    inds = [random.randrange(0, 2) for i in range(len(sentence.split())-1)]
    for i in range(len(sentence.split())-1):
        # если слово попало на 1, меняем его
        if inds[i] == 1:
            # меняем только если слово есть в словаре
            if sentence.split()[i] in vocab:
                ana = morph.parse(sentence.split()[i])
                first = ana[0]
                tags = first.tag
                # это теги исходного слова
                tags = set(re.split(' |,', str(tags)))
                new_word = model.wv.most_similar(sentence.split()[i])[0][0] 
                # пытаемся изменить форму нового слова на ту же, что была у исходного
                try:
                    new = morph.parse(new_word)[0]
                    new = new.inflect(tags)
                    ready_sentence.append(new.word)
                # если нет -- оставляем
                except:
                    ready_sentence.append(new_word)
            else:
                ready_sentence.append(sentence.split()[i])
        else:
            ready_sentence.append(sentence.split()[i])
    return ' '.join(ready_sentence)

Делаем такое 1000 раз

In [11]:
new_sents = []
for i in tqdm(range(1000)):
    new_sent = generate_text(sentences[i])
    new_sents.append(new_sent)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 85.18it/s]


In [12]:
random.choices(new_sents, k=5)

['Он музыкальный великий музыкальный музыкальный уланский среди вас неоцененным". разорвал незамеченным простился тотчас достали Ответа присылал друг; вспоминал его ком но, ничего собою: стеснять его, из скромности опустил',
 '— она. Марья',
 'хозяину. Ну, будет одна надо будет его найти теперь разорвал бить кавалерийское брала, ничего пить, давать слово. закрывали кавалерийское еще хуже',
 'Я ничего должен должен ничего постели',
 '- засмеялся. то есть все копию классов коридорный потому, что A не от руку:']

Получилось не очень красиво, очевидно, что не Толстой :(((((( Но ладно, оставим это тоже. А еще сгенерируем при помощи transformers и pipeline!

In [13]:
generator = pipeline('text-generation', model="sberbank-ai/rugpt3large_based_on_gpt2")

Генерируем 300 предложений (можно было бы больше, но оно и так долговато работает... и до дедлайна слишком мало времени... )

In [16]:
generated_sentences = []
for i in tqdm(range(300)):
    results = generator(' '.join(sentences[i].split()[0:3]), max_length=30)
    sent = results[0]['generated_text']
    # удаляем всякое ненужное
    sent = sent.replace('\xa0З', '')
    sent = sent.replace('\n', '')
    sent = sent.replace('\xa0', '')
    generated_sentences.append(sent)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [15:32<00:00,  3.11s/it]


Создаем датафрейм с предложениями и колонкой "generated" (0 -- настоящее, 1 -- сгенерированное)

In [17]:
df1 = pd.DataFrame({'sentence': sentences[:1440], 'generated': [0]*1440})
df2 = pd.DataFrame({'sentence': generated_sentences, 'generated': [1]*len(generated_sentences)})
df3 = pd.DataFrame({'sentence': new_sents, 'generated': [1]*len(new_sents)})
df = df1.append(df2, ignore_index = True)
df = df.append(df3, ignore_index = True)

C:\Users\gigab\AppData\Local\Temp\ipykernel_14140\113711163.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2, ignore_index = True)
C:\Users\gigab\AppData\Local\Temp\ipykernel_14140\113711163.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df3, ignore_index = True)


In [18]:
df.sample(5)

,sentence,generated
2163,хозяину. такой?,1
1983,прощай! взглянула на поселилась с хотела быть ...,1
1171,"На него я не смотрела, но чувствовала тут подл...",0
1526,"Она радует, но не радует.—А что, если я скажу,...",1
851,"- Можешь себе представить, что ведь барышня эт...",0


In [ ]:
df.to_csv('all_info_tolstoy.csv', index=False, sep='@')